In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import imdb
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Conv2DTranspose,MaxPooling1D, Embedding,Input, Conv1D, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from tensorflow.keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils
from keras.preprocessing import sequence

In [2]:
top_words = 1000
# Now we split our data-set into training and test data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


model = Sequential()
model.add(Embedding(1000, 32, input_length=max_words))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
# model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu',name="d1"))

model.add(Dense(1, activation='sigmoid', name="d2"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           32000     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 64)           6208      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 500, 64)           12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
d1 (Dense)                   (None, 256)               4096256   
_________________________________________________________________
d2 (Dense)                   (None, 1)                

In [5]:
# Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
196/196 [==============================] - 34s 22ms/step - loss: 0.5094 - accuracy: 0.7032 - val_loss: 0.3254 - val_accuracy: 0.8619
Epoch 2/10
196/196 [==============================] - 4s 22ms/step - loss: 0.3031 - accuracy: 0.8729 - val_loss: 0.2979 - val_accuracy: 0.8727
Epoch 3/10
196/196 [==============================] - 4s 20ms/step - loss: 0.2754 - accuracy: 0.8846 - val_loss: 0.2887 - val_accuracy: 0.8769
Epoch 4/10
196/196 [==============================] - 4s 20ms/step - loss: 0.2450 - accuracy: 0.9013 - val_loss: 0.2923 - val_accuracy: 0.8777
Epoch 5/10
196/196 [==============================] - 4s 20ms/step - loss: 0.2045 - accuracy: 0.9172 - val_loss: 0.3251 - val_accuracy: 0.8645
Epoch 6/10
196/196 [==============================] - 4s 20ms/step - loss: 0.1621 - accuracy: 0.9369 - val_loss: 0.3165 - val_accuracy: 0.8726
Epoch 7/10
196/196 [==============================] - 4s 20ms/step - loss: 0.1131 - accuracy: 0.9586 - val_loss: 0.3828 - val_accuracy: 0.866

In [6]:
scores = model.evaluate(X_test, y_test, verbose=1)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 2s 3ms/step - loss: 0.6618 - accuracy: 0.8603
Accuracy: 86.03%


In [38]:
teacher_WO_Softmax = Model(model.input, model.get_layer('d1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
# val_dense = teacher_WO_Softmax.predict(X_val)


s1Train=train_dense[:,:128]
s2Train=train_dense[:,128:]


In [39]:
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=(500,))
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile('adam',loss=['binary_crossentropy','mse'],loss_weights=[0.5,0.5],metrics=['accuracy'])

    return gan

In [40]:
def build_sdiscriminator():
    
    input2 = Input(shape=(128,),name='input')
    inp=Dense(64)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    conv3 = Dense(128,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    b_n=Dropout(0.25)(b_n)
    conv3 = Dense(256,activation='relu')(b_n)
    b_n = BatchNormalization()(conv3)
    conv4 = Dense(512,activation='relu')(b_n)
    b_n = BatchNormalization()(conv4)
    dense = Dense(1,activation='sigmoid')(b_n)
    output2=Dense(128)(b_n)
    disc = Model(input2,[dense,output2])          
    disc.compile(Adam(lr=0.0002,beta_1=0.5),loss=['binary_crossentropy','mse',],metrics=['accuracy'])


    return disc

In [56]:
def define_model(name):
  model = Sequential()
  model.add(tf.keras.layers.Embedding(1000, 10, input_length=500,name=name))
  model.add(Dropout(0.2))
  model.add(Conv1D(128, 3,strides=3,
                  activation='relu'))
  model.add(MaxPooling1D())
  model.add(Conv1D(64, 3,strides=3,
                  activation='relu'))
  model.add(MaxPooling1D())

  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu',name=name+"req"))
  # compile model
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  return model

In [57]:
define_model("s1").summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Embedding)               (None, 500, 10)           10000     
_________________________________________________________________
dropout_63 (Dropout)         (None, 500, 10)           0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 166, 128)          3968      
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 83, 128)           0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 27, 64)            24640     
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 13, 64)            0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 832)             

In [58]:
def training(generator,discriminator,gan,features,epo=20):
    # Setup Models here
    BATCH_SIZE = 128
    # discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
            
       
            # Write your code here
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [59]:
s1=define_model("s1")
d1 = build_sdiscriminator()
gan1 = build_gan(s1,d1)

s2=define_model("s2")
d2 = build_sdiscriminator()
gan2 = build_gan(s2,d2)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [60]:
s1=training(s1,d1,gan1,s1Train,20)
s2=training(s2,d2,gan2,s2Train,20)

195/195 [==============================] - 10s 40ms/step
Epoch: 1 | Discriminator Loss: 1.428574 | Generator Loss: 0.478447 | 
195/195 [==============================] - 8s 38ms/step
Epoch: 2 | Discriminator Loss: 0.949632 | Generator Loss: 0.426687 | 
195/195 [==============================] - 7s 38ms/step
Epoch: 3 | Discriminator Loss: 0.858708 | Generator Loss: 0.406747 | 
195/195 [==============================] - 7s 38ms/step
Epoch: 4 | Discriminator Loss: 0.836815 | Generator Loss: 0.400708 | 
195/195 [==============================] - 7s 38ms/step
Epoch: 5 | Discriminator Loss: 0.823548 | Generator Loss: 0.384011 | 
195/195 [==============================] - 7s 38ms/step
Epoch: 6 | Discriminator Loss: 0.816378 | Generator Loss: 0.388878 | 
195/195 [==============================] - 8s 40ms/step
Epoch: 7 | Discriminator Loss: 0.812128 | Generator Loss: 0.405557 | 
195/195 [==============================] - 8s 40ms/step
Epoch: 8 | Discriminator Loss: 0.808839 | Generator Loss: 0.4

In [61]:
o1=s1.get_layer("s1req").output
o2=s2.get_layer("s2req").output

out = tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(out)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)

mm2=Model([s1.get_layer("s1").input,
           s2.get_layer("s2").input],
          output3)
my_weights=model.get_layer('d2').get_weights()
mm2.get_layer('d11').set_weights(my_weights)

In [62]:
i=0
for l in mm2.layers[:len(mm2.layers)-1]:
    l.trainable=False

In [63]:
adam = Adam(0.0002,0.5,0.999)

mm2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [64]:
batch_size = 128
mm2_history=mm2.fit([X_train,X_train], y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_test,X_test], y_test))

Epoch 1/10
196/196 [==============================] - 3s 11ms/step - loss: 1.3101 - accuracy: 0.6118 - val_loss: 0.3931 - val_accuracy: 0.8239
Epoch 2/10
196/196 [==============================] - 2s 10ms/step - loss: 0.6695 - accuracy: 0.7385 - val_loss: 0.3818 - val_accuracy: 0.8278
Epoch 3/10
196/196 [==============================] - 2s 10ms/step - loss: 0.5092 - accuracy: 0.7888 - val_loss: 0.3754 - val_accuracy: 0.8301
Epoch 4/10
196/196 [==============================] - 2s 10ms/step - loss: 0.4440 - accuracy: 0.8091 - val_loss: 0.3759 - val_accuracy: 0.8295
Epoch 5/10
196/196 [==============================] - 2s 10ms/step - loss: 0.4021 - accuracy: 0.8208 - val_loss: 0.3755 - val_accuracy: 0.8306
Epoch 6/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3911 - accuracy: 0.8254 - val_loss: 0.3768 - val_accuracy: 0.8306
Epoch 7/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3812 - accuracy: 0.8296 - val_loss: 0.3746 - val_accuracy: 0.8322

In [65]:
# 1 student evaluate model
_, acc = mm2.evaluate([X_test,X_test], y_test, verbose=1)
print('> %.2f' % (acc * 100.0))

782/782 [==============================] - 2s 3ms/step - loss: 0.3736 - accuracy: 0.8344
> 83.44


In [66]:
mm2.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s1 (Embedding)                  (None, 500, 10)      10000       s1_input[0][0]                   
__________________________________________________________________________________________________
s2 (Embedding)                  (None, 500, 10)      10000       s2_input[0][0]                   
___________________________________________________________________________________________

## FF

In [67]:
s1= define_model("s1")
s2= define_model("s2")

In [68]:
batch_size = 128
mm_history=s1.fit(X_train, s1Train, batch_size=batch_size, epochs=10,verbose=1)
mm2_history=s2.fit(X_train, s2Train, batch_size=batch_size, epochs=10,verbose=1)

Epoch 1/10
196/196 [==============================] - 2s 8ms/step - loss: 0.3267 - accuracy: 0.8862
Epoch 2/10
196/196 [==============================] - 1s 7ms/step - loss: 0.2659 - accuracy: 0.9005
Epoch 3/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1875 - accuracy: 0.9005
Epoch 4/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1736 - accuracy: 0.9005
Epoch 5/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1669 - accuracy: 0.9006
Epoch 6/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1630 - accuracy: 0.9005
Epoch 7/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1610 - accuracy: 0.9005
Epoch 8/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1576 - accuracy: 0.9006
Epoch 9/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1578 - accuracy: 0.9006
Epoch 10/10
196/196 [==============================] - 1s 7ms/step - loss: 0.1553 - accuracy: 0.9004

In [69]:
o1=s1.get_layer("s1req").output
o2=s2.get_layer("s2req").output
out = tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(out)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)
# output4=Activation('softmax')(output3)
mm=Model([s1.get_layer("s1").input,s2.get_layer("s2").input], output3)
my_weights=model.get_layer('d2').get_weights()
mm.get_layer('d11').set_weights(my_weights)

In [70]:
i=0
for l in mm.layers[:len(mm.layers)-1]:
    l.trainable=False

In [71]:
adam = Adam(0.0002,0.5,0.999)

mm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [72]:
batch_size = 128
mm_history=mm.fit([X_train,X_train], y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_test,X_test], y_test))

Epoch 1/10
196/196 [==============================] - 3s 11ms/step - loss: 0.5516 - accuracy: 0.8514 - val_loss: 0.4479 - val_accuracy: 0.8382
Epoch 2/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3744 - accuracy: 0.8490 - val_loss: 0.3692 - val_accuracy: 0.8379
Epoch 3/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3491 - accuracy: 0.8493 - val_loss: 0.3645 - val_accuracy: 0.8373
Epoch 4/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3429 - accuracy: 0.8524 - val_loss: 0.3640 - val_accuracy: 0.8373
Epoch 5/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3357 - accuracy: 0.8556 - val_loss: 0.3644 - val_accuracy: 0.8373
Epoch 6/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3344 - accuracy: 0.8545 - val_loss: 0.3639 - val_accuracy: 0.8374
Epoch 7/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3348 - accuracy: 0.8558 - val_loss: 0.3646 - val_accuracy: 0.8372

In [73]:
_, acc = mm.evaluate([X_test,X_test], y_test, verbose=1)

782/782 [==============================] - 2s 3ms/step - loss: 0.3634 - accuracy: 0.8389
